[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/bae-v3/blob/main/part4/ch10/p10_type2.ipynb)

In [2]:
# 문제정의
# 평가: RMSE
# target: 총가스사용량
# 최종파일: result.csv(컬럼 1개 pred)

# 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bae-v3/main/part4/ch10/gas_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bae-v3/main/part4/ch10/gas_test.csv")

# 탐색적 데이터 분석(EDA)
print("===== 데이터 크기 =====")
print("Train Shape:", train.shape)
print("Test Shape:", test.shape)

print("\n ===== 데이터 샘플 =====")
print(train.head())

print("\n ===== 데이터 정보(자료형) =====")
print(train.info())

print("\n ===== train object컬럼 unique 수 =====")
print(train.describe(include='object'))

print("\n ===== test object컬럼 unique 수 =====")
print(test.describe(include='object'))

print("\n ===== train 결측치 수 =====")
print(train.isnull().sum().sum())

print("\n ===== test 결측치 수 =====")
print(test.isnull().sum().sum())

print("\n ===== target 기술 통계 =====")
print(train['총가스사용량'].describe())

===== 데이터 크기 =====
Train Shape: (3196, 6)
Test Shape: (1476, 5)

 ===== 데이터 샘플 =====
  시군구명  생활및판매  공공문화  복지의료  업무오락체육   총가스사용량
0  구로구      2     0     0       0   9077.8
1  구로구      6     0     1       2  10105.5
2  구로구     27     0     0       0   8603.6
3  구로구      2     0     0       0  11076.8
4  구로구     83     0     1      19  10781.4

 ===== 데이터 정보(자료형) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3196 entries, 0 to 3195
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시군구명    3196 non-null   object 
 1   생활및판매   3196 non-null   int64  
 2   공공문화    3196 non-null   int64  
 3   복지의료    3196 non-null   int64  
 4   업무오락체육  3196 non-null   int64  
 5   총가스사용량  3196 non-null   float64
dtypes: float64(1), int64(4), object(1)
memory usage: 149.9+ KB
None

 ===== train object컬럼 unique 수 =====
        시군구명
count   3196
unique    21
top      강남구
freq     715

 ===== test object컬럼 unique 수 =====
        시군구명
count  

In [3]:
# 데이터
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bae-v3/main/part4/ch10/gas_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bae-v3/main/part4/ch10/gas_test.csv")


# 데이터 전처리
target = train.pop('총가스사용량')

# 원핫 인코딩
# print("인코딩 전:", train.shape, test.shape)
train = pd.get_dummies(train)
test = pd.get_dummies(test)
# print("인코딩 후:", train.shape, test.shape)


# 검증데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# RMSE (Root Mean Squared Error)
from sklearn.metrics import root_mean_squared_error

# 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
print("랜덤포레스트:",root_mean_squared_error(y_val, pred))

# LightGBM
import lightgbm as lgb
lg = lgb.LGBMRegressor(random_state=0, verbose=-1)
lg.fit(X_tr, y_tr)
pred = lg.predict(X_val)
print("lightgbm:",root_mean_squared_error(y_val, pred))

# 최종 제출 파일
pred = rf.predict(test)
result = pd.DataFrame({'pred':pred})
result.to_csv("result.csv", index=False)

랜덤포레스트: 960.485846380754
lightgbm: 1064.8095758723994


In [4]:
# 1. pred 행의 수와 test의 행의 수 비교
print("pred: ",pred.shape) # test 행의 수: 1476

# 2. 생성한 csv 확인
print(pd.read_csv("result.csv").head())

pred:  (1476,)
           pred
0   9787.863633
1  10770.220091
2   9137.436055
3   9787.863633
4  11234.287608


[심화] 결측치(타겟: 0)을 제거한다면?

In [5]:
# 데이터
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bae-v3/main/part4/ch10/gas_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bae-v3/main/part4/ch10/gas_test.csv")

# 데이터 전처리
print("제거 전:", train.shape)
cond = train['총가스사용량'] == 0
train = train[~cond]
print("제거 후:", train.shape)

target = train.pop('총가스사용량')

# 원핫 인코딩
# print("인코딩 전:", train.shape, test.shape)
train = pd.get_dummies(train)
test = pd.get_dummies(test)
# print("인코딩 후:", train.shape, test.shape)


# 검증데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# RMSE (Root Mean Squared Error)
from sklearn.metrics import root_mean_squared_error

# 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
print("랜덤포레스트:",root_mean_squared_error(y_val, pred))

# LightGBM
import lightgbm as lgb
lg = lgb.LGBMRegressor(random_state=0, verbose=-1)
lg.fit(X_tr, y_tr)
pred = lg.predict(X_val)
print("lightgbm:",root_mean_squared_error(y_val, pred))

# 최종 제출 파일
pred = rf.predict(test)
result = pd.DataFrame({'pred':pred})
result.to_csv("result.csv", index=False)

제거 전: (3196, 6)
제거 후: (3139, 6)
랜덤포레스트: 673.8026644431536
lightgbm: 741.9117749289708
